# How to generate the tabular files for the A=45-60 nuclei

In the following we will explain the process of formatting the
electron-capture / beta decay rates, of  nuclei that belong to 
the iron group, into a table format that can be read by pynucastro.
 The rates we will consider are published here:

https://www.sciencedirect.com/science/article/pii/S0092640X01908654

and their accurate description can be found here:

https://iopscience.iop.org/article/10.1088/1402-4896/ac8dcb

In pynucastro the tabular rates, are read in a 2-step process.

In [37]:
import numpy as np
import scipy.constants as scp
import re

from pynucastro.nucdata.elements import Element, PeriodicTable

In [38]:
def abbreviation(entry):

    e = re.match(r"([A-Za-z]+)([0-9]*)", entry)
    element = e.group(1).lower()
    mass_number = str(int(e.group(2)))
    el = mass_number + element

    if el == "1n":
        el = "n"

    if el == "1h":
        el = "p"

    return el

In [39]:
eV_to_J, _, _ =scp.physical_constants['electron volt-joule relationship']
MeV_to_eV = 1.0e6
J_to_erg = 1.0e7

MeV_to_erg = MeV_to_eV * eV_to_J * J_to_erg 
T9_to_K = 1.0e9


with open("1-s2.0-S0092640X01908654-mmc01.txt", "r") as file:

    #get rid of the header
    for _ in range(15):
        file.readline()
    
    while True:

        line1 = file.readline()
        line2 = file.readline()

        if not line1:
            break

        line1 = line1.strip().split("=")
        line2 = line2.strip().split("=")

        el1_line = line1[0].split()
        el2_line = line2[0].split()

        el1 = abbreviation(el1_line[-2])
        el2 = abbreviation(el2_line[-2])

        Q = float(line1[-1])

        labels = file.readline()
        _ = labels = file.readline()

        with open(el1+"-"+el2+"_electroncapture"+".dat", "w") as output1, open(el2+"-"+el1+"_betadecay"+".dat", "w") as output2:

            output1.write(f"!{el1} -> {el2}, e- capture\n")
            output1.write(f"!Q={Q} MeV\n")
            output1.write(f"!\n")
            output1.write(f"!log(rhoY)    log(temp)     mu              dQ    Vs     log(e-cap-rate)        nu-energy-loss gamma-energy\n")
            output1.write(f"!g/cm^3       log(K)        1/s             erg   erg    log(1/s)                    erg/s          erg/s\n")

            output2.write(f"!{el2} -> {el1}, beta-decay\n")
            output2.write(f"!Q={-Q} MeV\n")
            output2.write(f"!\n")
            output2.write(f"!log(rhoY)    log(temp)     mu             dQ    VS     log(beta-decay-rate)   nu-energy-loss   gamma-energy\n")
            output2.write(f"!log(g/cm^3)  log(K)        1/s            erg   erg    log(1/s)               erg/s            erg/s\n")
            while True:
                line = file.readline().strip()

                if line == "end":
                    break

                data_str = line.split()
                temp = np.log10(float(data_str[0]) * T9_to_K)
                rho = float(data_str[1])
                mu_e = float(data_str[2]) * MeV_to_erg
                dq = 0.0
                vs = 0.0
                lbeta_p = 10.0**float(data_str[3])
                leps_m = float(data_str[4])
                lrnu = 10.0**float(data_str[5]) * MeV_to_erg
                lbeta_m = float(data_str[6])
                leps_p = 10.0**float(data_str[7])
                lrnu_bar = 10.0**float(data_str[8]) * MeV_to_erg
                gamma = 0.0

                output1.write(f"{rho:<12.6f}  {temp:<12.6f} {mu_e:>14.6e} {dq:>5.2f}  {vs:<6.2f} {leps_m:<22.6f} {lrnu:<13.6e}  {gamma:<6.2f}\n")
                output2.write(f"{rho:<12.6f}  {temp:<12.6f} {mu_e:>14.6e} {dq:>5.2f}  {vs:<6.2f} {lbeta_m:<22.6f} {lrnu_bar:<15.6e}  {gamma:<6.2f}\n")